In [1]:
pip install transformers[torch] accelerate==0.20.1

  Obtaining dependency information for accelerate==0.20.1 from https://files.pythonhosted.org/packages/77/a8/af1e480814d0cccfe60def63471841a365dbd6f94e2d308d9bc1e3db2da2/accelerate-0.20.1-py3-none-any.whl.metadata
INFO: pip is looking at multiple versions of transformers[torch] to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/0f/12/d8e27a190ca67811f81deea3183b528d9169f10b74d827e0b9211520ecfa/transformers-4.36.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/92/ba/cfff7e01f7070d9fca3964bf42b2257b86964c3e6763b8d5435436cc1d77/transformers-4.35.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 5.7 MB/s eta 0:00:00
  Obtaining dependency in

In [2]:
pip show accelerate

Name: accelerate
Version: 0.20.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: catalyst
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 937.1 kB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.1
    Uninstalling fsspec-2023.12.1:
      Successfully uninstalled fsspec-2023.12.1
  Attempting uninstall:

In [5]:
import os
import pandas as pd
import numpy as np
from datasets import Dataset,DatasetDict
from transformers import DataCollatorWithPadding
import evaluate
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [6]:
data = pd.read_csv("/kaggle/input/dataset-cpu/train.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [8]:
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
df_toxic = data.copy()
df_toxic['is_toxic'] = df_toxic[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].apply(lambda row: any(row), axis=1).astype(int)
df_toxic = df_toxic.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1)

In [10]:
df_toxic

,id,comment_text,is_toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [11]:
df_toxic = df_toxic.drop('id', axis=1)

In [12]:
df_toxic.head(5)

,comment_text,is_toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [13]:
df_toxic['is_toxic'].value_counts()

is_toxic
0    143346
1     16225
Name: count, dtype: int64

In [14]:
df_toxic_balanced = pd.concat([
    df_toxic[df_toxic['is_toxic'] == 0].sample(2000, random_state=42),
    df_toxic[df_toxic['is_toxic'] == 1].sample(2000, random_state=42)
])

df_toxic_balanced = df_toxic_balanced.sample(frac=1, random_state=42)

print(df_toxic_balanced['is_toxic'].value_counts())

is_toxic
0    2000
1    2000
Name: count, dtype: int64


In [15]:
df_toxic_balanced.dtypes

comment_text    object
is_toxic         int64
dtype: object

In [16]:
df_toxic_balanced = df_toxic_balanced.rename(columns={'is_toxic': 'label'})

In [17]:
df_toxic_balanced.dtypes

comment_text    object
label            int64
dtype: object

In [18]:
df_toxic_balanced['label'] = df_toxic_balanced['label'].map({0: 'Not Toxic', 1: 'Toxic'})

In [19]:
print(df_toxic_balanced.dtypes)
df_toxic_balanced.head(5)

comment_text    object
label           object
dtype: object


,comment_text,label
49475,No offence taken. I actually found your commen...,Not Toxic
4095,"I swear, you're extremely stupid and oblivious...",Toxic
102839,Do not dump identical text at multiple article...,Not Toxic
102223,"""\n\nI don't edit Wikipedia as I am no longer ...",Toxic
121720,THE ACLU IS ON YOUR ASS! \n\nThis latino will ...,Toxic


In [20]:
id2label = {0: "Not Toxic", 1: "Toxic"}
label2id = {"Not Toxic": 0, "Toxic": 1}
df_toxic_balanced["label"] = df_toxic_balanced["label"].apply(lambda x: label2id[x])

In [21]:
ds = Dataset.from_pandas(df_toxic_balanced)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [22]:
ds

Dataset({
    features: ['comment_text', 'label', '__index_level_0__'],
    num_rows: 4000
})

In [24]:
pretrained_model = 'bert-base-uncased'

In [25]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast =True) # False

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
def preprocess(x):
    return tokenizer(x["comment_text"], truncation=True, max_length=128)

In [27]:
tok_ds = ds.map(preprocess, batched=True)
tok_ds

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Dataset({
    features: ['comment_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4000
})

In [28]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length')

In [29]:
dataset = tok_ds.train_test_split(test_size=0.3)

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2800
    })
    test: Dataset({
        features: ['comment_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [31]:
tok_train_dataset = dataset["train"]
tok_test_dataset = dataset["test"]

In [32]:
accuracy = evaluate.load("accuracy")

In [33]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [34]:
def compute_metrics(pred):
    labels = pred.label_ids.float()
    preds = pred.predictions.argmax(-1).float()
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=2, id2label=id2label, label2id=label2id, output_attentions=True)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [36]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    fp16=True,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_train_dataset,
    eval_dataset=tok_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=350, training_loss=0.24226475306919643, metrics={'train_runtime': 227.3968, 'train_samples_per_second': 24.627, 'train_steps_per_second': 1.539, 'total_flos': 1473421910016000.0, 'train_loss': 0.24226475306919643, 'epoch': 2.0})

In [38]:
trainer.save_model("comments_model_bert")

In [53]:
model_save_path = '/kaggle/working/comments_model_bert.pkl'

with open(model_save_path, 'wb') as model_file:
    pickle.dump(model, model_file)

In [54]:
model_pickle_path = '/kaggle/working/comments_model_bert.pkl'

with open(model_pickle_path, 'rb') as model_file:
    model = pickle.load(model_file)

In [39]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
 
model_path = "/kaggle/working/comments_model_bert"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
 
pipeline =  TextClassificationPipeline(model=model, tokenizer=tokenizer)
print(pipeline("You are beautiful"))
print(pipeline("You are ugly"))

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'Not Toxic', 'score': 0.9476667046546936}]
[{'label': 'Toxic', 'score': 0.9899339079856873}]


In [51]:
print(pipeline("you're so so"))
print(pipeline("I wish you were dead"))
print(pipeline("you're gorgeous"))

[{'label': 'Toxic', 'score': 0.5880172252655029}]
[{'label': 'Toxic', 'score': 0.748916506767273}]
[{'label': 'Not Toxic', 'score': 0.7389889359474182}]


In [41]:
import json

config_path = "/kaggle/working/comments_model_bert/config.json"

with open(config_path, 'r') as config_file:
    config = json.load(config_file)

config


{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForSequenceClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'Not Toxic', '1': 'Toxic'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'Not Toxic': 0, 'Toxic': 1},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'output_attentions': True,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'problem_type': 'single_label_classification',
 'torch_dtype': 'float32',
 'transformers_version': '4.29.2',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}